In [1]:
import os
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [3]:
def check_if_character(line, punct_set):
    if "(" in line:
        line=line[:line.index("(")].strip()
    elif "{" in line:
        line=line[:line.index("{")].strip()
    elif '[' in line:
        line=line[:line.index('[')].strip()
    if line.upper()==line and line.isupper() and line[-1] not in punct_set and line.count(' ')<4:
        if "/" in line:
            line=line[:line.index('/')-1].strip()
        return True, line
    else:
        return False, ""

        


In [22]:
unacceptable_starters=["VOICE (cont'd)", "VOICE (CONT'D)", "VOICE OVER (CONT'D)", "VOICE OVER (cont'd)", "DISSOLVE", "CUT", "CUT TO", 'FADE', 'FADE OUT', 'FADE IN', 'PAN', 'CONTINUED', "CONT'D", '', ' ', "VOICE", "VOICE OVER", 'CUT TO', 'DISSOLVE TO', 'THE END', 'FADE TO BLACK', "DISSOLVE TO:", "CUT TO:", "FADE TO:"]

test_set=["blackdahliathe_dialog.txt", "basicinstinct_dialog.txt", "basic_dialog.txt","girlwiththedragontattoothe_dialog.txt", "manhattanmurdermystery_dialog.txt", 'afewgoodmen_dialog.txt', "8mm_dialog.txt", "backdraft_dialog.txt"]

movie_character_dict={}
punct_remover=str.maketrans('','', string.punctuation)

for movie in test_set:
    path=os.path.join("scripts", "samples", movie)

    with open(path, "r", encoding="ISO-8859-1") as f:
        movie_script=f.readlines()
        title=""
        start_of_content=False
        current_character=""       
      
        for line in movie_script:
            line=line.strip()

            #checks for the title and the first character, otherwise skips the header stuff
            if not start_of_content:
                line=line.strip('"')
                if title=="" and line.translate(punct_remover)==line and line not in unacceptable_starters:
                    title=line
                    movie_character_dict[title]={}
                is_character, current_character = check_if_character(line, string.punctuation)
                if is_character and (current_character not in unacceptable_starters):
                    start_of_content=True
                    movie_character_dict[title][current_character]=[]
            
            #if in the middle of the script and you have a current character, check if this line is a new character otherwise add the dialogue to this character's list
            else:
                is_new_character, new_character_name=check_if_character(line, string.punctuation)
                if is_new_character:
                    current_character=new_character_name
                    if new_character_name not in unacceptable_starters and new_character_name not in movie_character_dict[title]:
                        movie_character_dict[title][new_character_name]=[]
                else:
                    if current_character in unacceptable_starters:
                        pass
                    else:
                        if len(line)>0 and line[0]!='(' and line[-1]!=')':
                            for word in line.split(' '):
                                if len(word)>0:
                                    movie_character_dict[title][current_character].append(word.strip(string.punctuation))
                            
            






In [12]:
amalgamated_dialogues=[]
for movie in movie_character_dict:
    for character in movie_character_dict[movie]:
        amalgamated_dialogues.append(movie_character_dict[movie][character])

In [36]:
movie_character_dict[''].keys()

dict_keys(['MORELL', 'VANGER', 'BLOMKVIST', 'REPORTER 1', 'REPORTER 2', 'NEWSCASTER', 'TV NEWSCASTER', 'WENNERSTROM ON TV', 'ERIKA', 'ARMANSKY', 'FRODE', 'SECRETAR', 'SALANDER', 'GREGER', 'ANNIKA', 'GREGOR', 'PERNILLA', 'DOCTOR', 'PLAGUE', 'HUSSEIN', 'NILSSON', 'DET. MORELL', 'YOUNGER VANGER', 'BJURMAN', 'LIV', 'MARTIN', 'CECILIA', 'TECH', 'SECRETARY', 'ANITA', 'HARRIET', 'TATTOO ARTIST', 'ANNOUNCER', 'REPORTER 3', 'PHOTO EDITOR', 'DET. ISAKSSON', 'BLOMVIST', 'ISABELLA', 'BIRGER', 'NURSE', 'HARRIE', 'UDDEVALLY DETECTIVE', 'UDDEVALLA DETECTIVE', 'TECH CLERK', 'CLERK 1', 'CLERK 2', 'FORSMAN', 'MILDRED', 'LINDGREN', 'HARALD', 'TRINITY', 'WENNERSTROM', 'REPORTER', 'FSA OFFICIAL', 'LAWYER', 'IMMIGRATION OFFICIAL', 'WAGNER', 'HASSELMAN', 'ZURICH REPORTER', 'ZURICH REPORTER ON TV', 'REPORTER ON TV', 'BARISTA', 'TAILOR'])

In [25]:
# movie_character_dict['The Black Dahlia']['LEE BLANCHARD']+=movie_character_dict['The Black Dahlia'].pop('LEE')
# movie_character_dict['The Black Dahlia']['ELLIS LOEW']+=movie_character_dict['The Black Dahlia'].pop('LOEW')
# movie_character_dict['BASIC INSTINCT']['THE I.A. MAN']+=movie_character_dict['BASIC INSTINCT'].pop('AN I.A. MAN')
# movie_character_dict['Basic']['DUNBAR']+=movie_character_dict['Basic'].pop('DUN BAR')
# movie_character_dict['Basic']['MUELLER']+=movie_character_dict['Basic'].pop('MUE:LLER')
# movie_character_dict['Basic']['MUELLER']+=['I', 'got', 'a','friend','with','an','in','at','the','PX','can','hook','you','up']
# movie_character_dict['Basic']['DUNBAR']+=['I', 'got', 'a','friend','with','an','in','at','the','PX','can','hook','you','up']
# movie_character_dict['Basic'].pop('MUELLER AND DUNBAR')